In [1]:
import pandas as pd
import numpy as np
import sagemaker
import joblib
import tarfile

from sagemaker.sklearn.model import SKLearnModel
from sagemaker import get_execution_role

# Initialize SageMaker session
session = sagemaker.Session()
bucket = session.default_bucket()
role = get_execution_role()


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
data = []
for _ in range(1000):
    temp = round(np.random.uniform(20, 40), 2)
    humidity = round(np.random.uniform(50, 90), 2)
    soil = round(np.random.uniform(5, 40), 2)
    irrigation = 1 if soil < 15 and temp > 30 else 0
    data.append([temp, humidity, soil, irrigation])

df = pd.DataFrame(data, columns=["temperature", "humidity", "soil_moisture", "irrigation_needed"])
df.to_csv("sensor_training_data.csv", index=False)
df.head()


,temperature,humidity,soil_moisture,irrigation_needed
0,30.82,65.12,31.41,0
1,20.22,74.94,32.23,0
2,22.79,86.17,19.16,0
3,29.28,79.74,36.73,0
4,22.56,53.00,32.58,0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load dataset
df = pd.read_csv("sensor_training_data.csv")
X = df[["temperature", "humidity", "soil_moisture"]]
y = df["irrigation_needed"]

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# Save with protocol 4 for Python 3.7 compatibility
joblib.dump(model, "irrigation_model.joblib", protocol=4)
print(" Model saved with protocol 4")


In [ ]:
import tarfile

with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("irrigation_model.joblib", arcname="irrigation_model.joblib")

print(" New model.tar.gz created")


In [ ]:
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()

s3_path = session.upload_data("model.tar.gz", bucket=bucket, key_prefix="models")
print(" Model uploaded to:", s3_path)


-------------------------